<a href="https://colab.research.google.com/github/AK-ayush/GPU_training/blob/master/GPGPU-101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Getting started with General purpose GPU computing in python

---
<div align="center"><img src="https://media.springernature.com/full/springer-static/image/art%3A10.1186%2Fs12859-017-1666-0/MediaObjects/12859_2017_1666_Fig2_HTML.gif" width="400"/></div>

*Left side- Thread organization*: Threads (red cubes) are organized in three-dimensional structures called blocks (yellow cubes), which belong to three-dimensional grid (green cube). The programmer must explicitly define the dimensions of blocks and grids. 

*Right side- Memory hierarchy*: In CUDA there are many different memories with different scopes. Each thread has two different kind of private memory: registers and local memories. Threads belonging to the same block can communicate through the shared memory, which has low access latency. The global memory suffers from high access latencies but it is accessible to all threads.

[Courtesy: BMC Bioinformatics](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-017-1666-0/figures/2)

---
<div align="center"><img src="http://numba.pydata.org/_static/numba-blue-horizontal-rgb.svg" width="400"/></div>

## Numba makes Python code fast

CUDA has an execution model unlike the traditional sequential model used for programming CPUs. In CUDA, the code you write will be executed by multiple threads at once (often hundreds or thousands). **Your solution will be modeled by defining a thread hierarchy of grid, blocks and threads**.

Numba’s CUDA support exposes facilities to declare and manage this hierarchy of threads. The facilities are largely similar to those exposed by NVidia’s CUDA C language.

```python
from numba import cuda
import numpy as np

# Kernel declaration:
@cuda.jit
def increment_by_one(d_array):
    # Thread id in a 1D block
    tx = cuda.threadIdx.x
    # Block id in a 1D grid
    ty = cuda.blockIdx.x
    # Block width, i.e. number of threads per block
    bw = cuda.blockDim.x
    # Compute flattened index inside the array
    pos = tx + ty * bw
    if pos < d_array.size:  # Check array boundaries
        d_array[pos] += 1

#input array initialization
h_array = np.arange(100)
d_array = cuda.to_device(h_array)

# A kernel is typically launched in the following way:
threadsperblock = 32
blockspergrid = (d_array.size + (threadsperblock - 1)) // threadsperblock
increment_by_one[blockspergrid, threadsperblock](d_array)
```
[Courtesy: Numba pydata](http://numba.pydata.org/numba-doc/latest/cuda/kernels.html)

---

<div align="center"><img src="https://raw.githubusercontent.com/cupy/cupy/master/docs/image/cupy_logo_1000px.png" width="400"/></div>

## CuPy : NumPy-like API accelerated with CUDA

```python
#cupy examples
import cupy as cp
import numpy as np

x_gpu = cp.array([1, 2, 3])
l2_gpu = cp.linalg.norm(x_gpu)

# move the data across host and device
x_cpu = np.array([1, 2, 3])
x_gpu = cp.asarray(x_cpu)  # move the data to the current GPU device.

x_cpu = cp.asnumpy(x_gpu)  # move the array to the host.
```
[Courtesy: CuPy Chainer](https://docs-cupy.chainer.org/en/stable/reference/index.html)

---
In this notebook, we are utilizing the GloVe word vectors dataset. Given a word, we are trying to find the nearest word from the rest of the words based on ecludian distance and cosine similarity respectively.

This can be extended to image embeddings and other problems utilising vector space model. 

In [0]:
import pandas as pd
import numpy as np
import math
import collections
import os

In [3]:
!nvidia-smi

Sun Dec  1 19:26:53 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [1]:
# download glove vectors from web
!wget http://nlp.stanford.edu/data/glove.6B.zip #--quiet
!unzip glove.6B.zip

--2019-12-01 19:13:03--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-12-01 19:13:03--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-12-01 19:13:03--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:
!head -1 glove.6B.100d.txt

the -0.038194 -0.24487 0.72812 -0.39961 0.083172 0.043953 -0.39141 0.3344 -0.57545 0.087459 0.28787 -0.06731 0.30906 -0.26384 -0.13231 -0.20757 0.33395 -0.33848 -0.31743 -0.48336 0.1464 -0.37304 0.34577 0.052041 0.44946 -0.46971 0.02628 -0.54155 -0.15518 -0.14107 -0.039722 0.28277 0.14393 0.23464 -0.31021 0.086173 0.20397 0.52624 0.17164 -0.082378 -0.71787 -0.41531 0.20335 -0.12763 0.41367 0.55187 0.57908 -0.33477 -0.36559 -0.54857 -0.062892 0.26584 0.30205 0.99775 -0.80481 -3.0243 0.01254 -0.36942 2.2167 0.72201 -0.24978 0.92136 0.034514 0.46745 1.1079 -0.19358 -0.074575 0.23353 -0.052062 -0.22044 0.057162 -0.15806 -0.30798 -0.41625 0.37972 0.15006 -0.53212 -0.2055 -1.2526 0.071624 0.70565 0.49744 -0.42063 0.26148 -1.538 -0.30223 -0.073438 -0.28312 0.37104 -0.25217 0.016215 -0.017099 -0.38984 0.87424 -0.72569 -0.51058 -0.52028 -0.1459 0.8278 0.27062


In [0]:
!wc -l glove.6B.100d.txt

400000 glove.6B.100d.txt


In [0]:
def name_dtype_gen(dim_size=50):
    col_types = collections.OrderedDict()
    col_types['word'] = 'str'
    for i in range(dim_size):
        col_types['dim_'+str(i+1)]=np.float32
    return col_types

col_dtypes = name_dtype_gen(100)

In [5]:
%%time 
df = pd.read_csv("glove.6B.100d.txt", delim_whitespace=True, names=col_dtypes.keys() , quoting=3) #ignore quoting
df = df.astype(col_dtypes) 
df.shape

CPU times: user 6.58 s, sys: 695 ms, total: 7.28 s
Wall time: 7.3 s


In [6]:
 df.head()

,word,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,dim_11,dim_12,dim_13,dim_14,dim_15,dim_16,dim_17,dim_18,dim_19,dim_20,dim_21,dim_22,dim_23,dim_24,dim_25,dim_26,dim_27,dim_28,dim_29,dim_30,dim_31,dim_32,dim_33,dim_34,dim_35,dim_36,dim_37,dim_38,dim_39,...,dim_61,dim_62,dim_63,dim_64,dim_65,dim_66,dim_67,dim_68,dim_69,dim_70,dim_71,dim_72,dim_73,dim_74,dim_75,dim_76,dim_77,dim_78,dim_79,dim_80,dim_81,dim_82,dim_83,dim_84,dim_85,dim_86,dim_87,dim_88,dim_89,dim_90,dim_91,dim_92,dim_93,dim_94,dim_95,dim_96,dim_97,dim_98,dim_99,dim_100
0,the,-0.038194,-0.244870,0.72812,-0.399610,0.083172,0.043953,-0.391410,0.334400,-0.57545,0.087459,0.287870,-0.06731,0.30906,-0.26384,-0.13231,-0.20757,0.33395,-0.338480,-0.31743,-0.48336,0.146400,-0.373040,0.34577,0.052041,0.449460,-0.469710,0.026280,-0.54155,-0.15518,-0.141070,-0.039722,0.28277,0.14393,0.234640,-0.31021,0.086173,0.20397,0.52624,0.171640,...,-0.24978,0.92136,0.034514,0.46745,1.10790,-0.193580,-0.074575,0.233530,-0.052062,-0.220440,0.057162,-0.15806,-0.307980,-0.416250,0.379720,0.150060,-0.532120,-0.205500,-1.25260,0.071624,0.70565,0.497440,-0.42063,0.26148,-1.5380,-0.30223,-0.073438,-0.283120,0.371040,-0.25217,0.016215,-0.017099,-0.389840,0.87424,-0.72569,-0.51058,-0.520280,-0.14590,0.82780,0.270620
1,",",-0.107670,0.110530,0.59812,-0.543610,0.673960,0.106630,0.038867,0.354810,0.06351,-0.094189,0.157860,-0.81665,0.14172,0.21939,0.58505,-0.52158,0.22783,-0.166420,-0.68228,0.35870,0.425680,0.190210,0.91963,0.575550,0.461850,0.423630,-0.095399,-0.42749,-0.16567,-0.056842,-0.295950,0.26037,-0.26606,-0.070404,-0.27662,0.158210,0.69825,0.43081,0.279520,...,-0.22080,0.18669,0.131770,0.15117,0.71310,-0.352150,0.913480,0.617830,0.709920,0.239550,-0.145710,-0.37859,-0.045959,-0.473680,0.238500,0.205360,-0.189960,0.325070,-1.11120,-0.363410,0.98679,-0.084776,-0.54008,0.11726,-1.0194,-0.24424,0.127710,0.013884,0.080374,-0.35414,0.349510,-0.722600,0.375490,0.44410,-0.99059,0.61214,-0.351110,-0.83155,0.45293,0.082577
2,.,-0.339790,0.209410,0.46348,-0.647920,-0.383770,0.038034,0.171270,0.159780,0.46619,-0.019169,0.414790,-0.34349,0.26872,0.04464,0.42131,-0.41032,0.15459,0.022239,-0.64653,0.25256,0.043136,-0.194450,0.46516,0.456510,0.685880,0.091295,0.218750,-0.70351,0.16785,-0.350790,-0.126340,0.66384,-0.25820,0.036542,-0.13605,0.402530,0.14289,0.38132,-0.122830,...,-0.55262,0.65000,0.086426,0.39012,1.06320,-0.353790,0.483280,0.346000,0.841740,0.098707,-0.242130,-0.27053,0.045287,-0.401470,0.113950,0.006223,0.036673,0.018518,-1.02130,-0.208060,0.64072,-0.068763,-0.58635,0.33476,-1.1432,-0.11480,-0.250910,-0.459070,-0.096819,-0.17946,-0.063351,-0.674120,-0.068895,0.53604,-0.87773,0.31802,-0.392420,-0.23394,0.47298,-0.028803
3,of,-0.152900,-0.242790,0.89837,0.169960,0.535160,0.487840,-0.588260,-0.179820,-1.35810,0.425410,0.153770,0.24215,0.13474,0.41193,0.67043,-0.56418,0.42985,-0.012183,-0.11677,0.31781,0.054177,-0.054273,0.35516,-0.302410,0.314340,-0.338460,0.717150,-0.26855,-0.15837,-0.474670,0.051581,-0.33252,0.15003,-0.129900,-0.54617,-0.378430,0.64261,0.82187,-0.080006,...,0.04885,0.78267,0.384970,0.42097,0.67882,0.103370,0.632800,-0.026595,0.586470,-0.443320,0.330570,-0.12022,-0.556450,0.073611,0.209150,0.433950,-0.012761,0.089874,-1.79910,0.084808,0.77112,0.631050,-0.90685,0.60326,-1.7515,0.18596,-0.506870,-0.702030,0.665780,-0.81304,0.187120,-0.018488,-0.267570,0.72700,-0.59363,-0.34839,-0.560940,-0.59100,1.00390,0.206640
4,to,-0.189700,0.050024,0.19084,-0.049184,-0.089737,0.210060,-0.549520,0.098377,-0.20135,0.342410,-0.092677,0.16100,-0.13268,-0.28160,0.18737,-0.42959,0.96039,0.139720,-1.07810,0.40518,0.505390,-0.550640,0.48440,0.380440,-0.002905,-0.349420,-0.099696,-0.78368,1.03630,-0.231400,-0.471210,0.57126,-0.21454,0.359580,-0.48319,1.087500,0.28524,0.12447,-0.039248,...,-0.34780,0.51621,-0.433870,0.36852,0.74573,0.072102,0.279310,0.925690,-0.050336,-0.858560,-0.135800,-0.92551,-0.339910,-1.039400,-0.067203,-0.213790,-0.476900,0.213770,-0.84008,0.052536,0.59298,0.296

In [7]:
df.shape

(400000, 101)

In [8]:
mat = df.loc[df['word']!='queen', df.columns!='word'].to_numpy() 
test_vec = df.loc[df['word']=='queen', df.columns!='word'].to_numpy()
df = df.loc[df['word']!='queen']
words_arr = df.word.to_numpy()
mat.shape, test_vec.shape

((399999, 100), (1, 100))

In [9]:
df.shape

(399999, 101)


### Ecludian Distance:
<div align="center"><img src="http://mines.humanoriented.com/classes/2010/fall/csci568/portfolio_exports/sphilip/images/euclid_eqn.gif" width="300"/></div>

In [0]:
def ecludean_dist(a,b, dim_size):
    summ = 0
    for i in range(dim_size):
        summ += ((a[i]-b[i])**2)
    return math.sqrt(summ)

### Cosine Similarity:
<div align="center"><img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/1d94e5903f7936d3c131e040ef2c51b473dd071d" width="400"/></div>

In [0]:
def dot(a, b, dim_size):
    summ = 0
    for i in range(dim_size):
        summ += (a[i]*b[i])
    return summ

def cosine_sim(a, b, dim_size):
    return dot(a,b, dim_size) / ( math.sqrt(dot(a, a, dim_size)) * math.sqrt(dot(b, b, dim_size)))

In [0]:
def find_nearest(mat, vec, dim_size, n):
    ret_eldn = np.array([ecludean_dist(vec, mat[idx], dim_size) for idx in range(n)])
    ret_csn = np.array([cosine_sim(vec, mat[idx], dim_size) for idx in range(n)])
    return ret_eldn, ret_csn

In [0]:
%%time 
ret_eldn, ret_csn = find_nearest(mat, test_vec[0], mat.shape[1], mat.shape[0])
i1, i2 = np.argpartition(ret_eldn, 5)[:5], np.argpartition(ret_csn, -5)[-5:]

CPU times: user 3min 50s, sys: 368 ms, total: 3min 50s
Wall time: 3min 50s


In [0]:
i1,i2

(array([ 691, 4241, 3217, 2790, 3672]), array([2790, 4241, 3217, 1142,  691]))

In [0]:
print("Given word: ", 'queen')
print("Nearest using Ecludian: ", words_arr[i1])
print("Nearest using Cosine: ", words_arr[i2])

Given word:  queen
Nearest using Ecludian:  ['king' 'princess' 'elizabeth' 'lady' 'victoria']
Nearest using Cosine:  ['lady' 'princess' 'elizabeth' 'royal' 'king']


In [0]:
del ret_eldn, ret_csn, i1, i2

In [0]:
from numba import cuda
import cupy as cp

In [0]:
os.environ["NUMBAPRO_NVVM"]="/usr/local/cuda/nvvm/lib64/libnvvm.so"
os.environ["NUMBAPRO_LIBDEVICE"]="/usr/local/cuda/nvvm/libdevice/"

In [0]:
dim_size=100

In [0]:
@cuda.jit(device=True)
def ecludean_dist(a,b, dim_size):
    summ = 0
    for i in range(dim_size):
        summ += ((a[i]-b[i])**2)
    return math.sqrt(summ)

In [0]:
@cuda.jit(device=True)
def dot(a, b, dim_size):
    summ = 0
    for i in range(dim_size):
        summ += (a[i]*b[i])
    return summ

@cuda.jit(device=True)
def cosine_sim(a, b, dim_size):
    return dot(a,b, dim_size) / ( math.sqrt(dot(a, a, dim_size)) * math.sqrt(dot(b, b, dim_size)) )

In [0]:
@cuda.jit('void(float32[:,:], float32[:], float32[:], float32[:], int32, int32)')
def find_nearest_gpu(d_mat, d_vec, ret_eldn, ret_csn, dim_size, n):
    idx = cuda.threadIdx.x + cuda.blockDim.x * cuda.blockIdx.x
    if idx < n:  
      ret_eldn[idx] = ecludean_dist(d_vec, d_mat[idx], dim_size)
      ret_csn[idx] = cosine_sim(d_vec, d_mat[idx], dim_size)

In [0]:
n = mat.shape[0]

d_mat = cp.asarray(mat)
d_vec = cp.asarray(test_vec[0])
ret_eldn = cp.zeros(shape=n, dtype=np.float32)  
ret_csn = cp.zeros(shape=n, dtype=np.float32) 

# d_mat = cuda.to_device(mat)
# d_vec = cuda.to_device(test_vecs)
# ret_eldn = cuda.device_array(shape=n, dtype=np.float32)
# ret_csn = cuda.device_array(shape=n, dtype=np.float32)

In [17]:
device = cuda.get_current_device()

tpb = 64#device.WARP_SIZE    #blocksize or thread per block
bpg = int(np.ceil((n)/tpb))  # block per grid
(tpb, bpg)

(64, 6250)

In [18]:
%%time
find_nearest_gpu[bpg,tpb](d_mat, d_vec, ret_eldn, ret_csn, dim_size, n)
cuda.synchronize()
i1, i2 = cp.argpartition(ret_eldn, 5)[:5], cp.argpartition(ret_csn, -5)[-5:]

CPU times: user 320 ms, sys: 5.05 ms, total: 325 ms
Wall time: 442 ms


In [19]:
i1, i2

(array([4241, 3217,  691, 2790, 3672]), array([2790, 1142, 3217,  691, 4241]))

In [20]:
print("Given word: ", 'queen')
print("Nearest using Ecludian: ", words_arr[i1.tolist()])
print("Nearest using Cosine: ", words_arr[i2.tolist()])

Given word:  queen
Nearest using Ecludian:  ['princess' 'elizabeth' 'king' 'lady' 'victoria']
Nearest using Cosine:  ['lady' 'royal' 'elizabeth' 'king' 'princess']
